<a href="https://colab.research.google.com/github/Kang-Beom-Seo/Korean-NLP-Practice/blob/main/%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%A0%84%EC%B2%98%EB%A6%AC_%ED%8C%A8%ED%82%A4%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git #전희원님이 개발한 한국어 띄어쓰기 패키지로
                                                               #띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지입니다.
                                                               #PyKoSpacing은 대용량 코퍼스를 학습하여 만들어진 띄어쓰기 딥 러닝 모델로 준수한 성능을 가지고 있습니다.

In [ ]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'

In [ ]:
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

In [ ]:
from pykospacing import spacing

kospacing_sent = spacing(new_sent) #띄어쓰기를 해준다.
print(sent)
print(kospacing_sent) #정확히 일치한다.

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

In [ ]:
spelled_sent = spell_checker.check(new_sent)

hanspell_sent = spelled_sent.checked #띄어쓰기도 같이 보정해준다.
print(hanspell_sent)
print(kospacing_sent) # 앞서 사용한 kospacing 패키지에서 얻은 결과

In [ ]:
!pip install soynlp

In [ ]:
!pip install konlpy

In [ ]:
import konlpy
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정')) #신조어는 제대로 형태소 분석기가 작동하지 않는다.

In [ ]:
import urllib.request
from soynlp import DoublespaceLineCorpus #soynlp는 학습 기반의 단어 토크나이저이므로 학습을 시켜야 한다.
from soynlp.word import WordExtractor

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

In [ ]:
# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

In [ ]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

In [ ]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

In [ ]:
word_score_table["반포한"].cohesion_forward

In [ ]:
word_score_table["반포한강"].cohesion_forward

In [ ]:
word_score_table["반포한강공"].cohesion_forward

In [ ]:
word_score_table["반포한강공원"].cohesion_forward #제일 높으므로, 하나의 단어로 판단하기에 제일 적절한 단어

In [ ]:
word_score_table["반포한강공원에"].cohesion_forward

In [ ]:
word_score_table["디스"].right_branching_entropy

In [ ]:
word_score_table["디스플"].right_branching_entropy #'디스' 다음에는 다양한 문자가 올 수 있으니까 1.63이라는 값을 가지는 반면
                                                   #'디스플'이라는 문자열 다음에는 다음 문자로 '레'가 오는 것이 너무나 명백하기 때문에 0이란 값을 가집니다.

In [ ]:
word_score_table["디스플레"].right_branching_entropy

In [ ]:
word_score_table["디스플레이"].right_branching_entropy #갑자기 증가하는 이유는 이 다음에 조사나 다른 단어 여러가지가 올수 있어서이다.
                                                       #이 값으로 단어를 구분하는 것이다.

In [ ]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores) #L토큰(주로 명사)+R토큰(주로 조사)에서 L토큰의 score가 높은걸 찾아서 그 기준으로 분리하는 Tokenizer이다.
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer #띄어쓰기가 안된 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아낸다.

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

In [ ]:
from soynlp.normalizer import *

In [ ]:
print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2)) #반복되는 이모티콘을 줄여주어서 불필요하게 사전이 커지는 걸 막는다.
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))

In [ ]:
print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2)) #이모티콘이 아니여도 불필요한 단어 제거가 가능하다.
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))

In [ ]:
pip install customized_konlpy

In [ ]:
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.') #이름이 분리가 되어버렸다.

In [ ]:
twitter.add_dictionary('은경이', 'Noun') #사용자 사전에 추가함.

In [ ]:
twitter.morphs('은경이는 사무실로 갔습니다.') #정상적으로 잘 분리함.